In [1]:
import cv2


In [2]:
print(cv2.__version__)

4.10.0


In [2]:
import cv2
cap = cv2.VideoCapture(0) # Initialize the video capture(0-default webcam)

# Check if camera is opened successfully
if not cap.isOpened():
    print("Cannot open camera.")
    exit()

# Loop to capture frames continuously
while True:
    ret, frame = cap.read() # Read a frame from camera

    # Breaking the loop if frame reading fails
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    # Display the frame in the window
    cv2.imshow('Real-time video feed', frame)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the window
cap.release()
cv2.destroyAllWindows()

In [2]:
import cv2
import numpy as np
import time

def is_in_restricted_zone(cx, cy, zone):
    """
    Check if a point (cx, cy) is within the restricted zone.
    """
    x1, y1, x2, y2 = zone
    return x1 <= cx <= x2 and y1 <= cy <= y2

# Initialize background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)

# Define the restricted zone (x1, y1, x2, y2)
restricted_zone = (100, 100, 00, 4004)

# Cooldown timer for alerts
last_alert_time = 0
alert_cooldown = 2  # seconds

# Start video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame for better performance (optional)
    frame = cv2.resize(frame, (640, 480))

    # Apply background subtraction
    fg_mask = bg_subtractor.apply(frame)

    # Apply thresholding to the mask to clean up noise
    _, thresh = cv2.threshold(fg_mask, 50, 255, cv2.THRESH_BINARY)

    # Remove noise using morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # Find contours of the motion
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw the restricted zone on the frame
    cv2.rectangle(frame, (restricted_zone[0], restricted_zone[1]),
                  (restricted_zone[2], restricted_zone[3]), (0, 0, 255), 2)

    motion_detected = False  # Flag to track motion in the restricted zone

    for contour in contours:
        if cv2.contourArea(contour) < 700:  # Ignore small movements
            continue

        # Calculate the bounding box and center of motion
        x, y, w, h = cv2.boundingRect(contour)
        cx, cy = x + w // 2, y + h // 2

        # Check if the motion is in the restricted zone
        if is_in_restricted_zone(cx, cy, restricted_zone):
            motion_detected = True
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, "MOTION DETECTED!", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Trigger alert with cooldown
            current_time = time.time()
            if current_time - last_alert_time > alert_cooldown:
                print("ALERT: Motion detected in restricted zone!")
                last_alert_time = current_time

    # If no motion is detected in the restricted zone, ensure the alert is off
    if not motion_detected:
        cv2.putText(frame, "No Motion in Restricted Zone", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    # Display the frames
    cv2.imshow("Live Feed", frame)
    cv2.imshow("Threshold Frame", thresh)

    # Break on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


ALERT: Motion detected in restricted zone!
ALERT: Motion detected in restricted zone!
ALERT: Motion detected in restricted zone!
ALERT: Motion detected in restricted zone!
ALERT: Motion detected in restricted zone!
ALERT: Motion detected in restricted zone!
ALERT: Motion detected in restricted zone!
ALERT: Motion detected in restricted zone!
ALERT: Motion detected in restricted zone!


In [32]:
# Import the necessary libraries
import cv2
import cv2 as cv
import numpy as np

# Define a dictionary mapping human body parts to their corresponding indices in the model's output
BODY_PARTS = {
    "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
    "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
    "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
    "LEye": 15, "REar": 16, "LEar": 17, "Background": 18
}

# Define a list of pairs representing the body parts that should be connected to visualize the pose
POSE_PAIRS = [
    ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
    ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
    ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
    ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
    ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"]
]

# Specify the input dimensions for the neural network
width = 368
height = 368
inWidth = width
inHeight = height

# Load the pre-trained OpenPose model from a file
net = cv.dnn.readNetFromTensorflow(r"E:\Optimizing Workplace Compliance and Safety\human-pose-estimation-opencv\graph_opt.pb")
thr = 0.2  # Set a confidence threshold for detecting keypoints

# Define a function to detect poses in an input frame
def poseDetector(frame):
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]

    # Prepare the input for the model by resizing and mean normalization
    net.setInput(cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :19, :, :]  # Extract the first 19 elements, corresponding to the body part keypoints

    # Ensure the number of detected body parts matches the predefined BODY_PARTS
    assert(len(BODY_PARTS) == out.shape[1])

    points = []  # Initialize a list to hold the detected keypoints
    # Iterate over each body part to find the keypoints
    for i in range(len(BODY_PARTS)):
        # Extract the heatmap for the current body part
        heatMap = out[0, i, :, :]
        # Find the point with the maximum confidence
        _, conf, _, point = cv.minMaxLoc(heatMap)
        # Scale the point's coordinates back to the original frame size
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]
        # Add the point to the list if its confidence is above the threshold
        points.append((int(x), int(y)) if conf > thr else None)

    # Draw lines and ellipses to represent the pose in the frame
    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        # Ensure the body parts are in the BODY_PARTS dictionary
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        # If both keypoints are detected, draw the line and keypoints
        if points[idFrom] and points[idTo]:
            cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
            cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
            cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

    t, _ = net.getPerfProfile()  # Optional: Retrieve the network's performance profile

    return frame  # Return the frame with the pose drawn

# Load an input image
input = cv.imread(r"E:\Optimizing Workplace Compliance and Safety\Data\pic2.jpg")
# Pass the image to the poseDetector function
output = poseDetector(input)
# Display the output image with the detected pose
cv2.imshow("Pose Estimation", output)
cv2.waitKey(0)
cv2.destroyAllWindows()